In [1]:
from functools import partial
import numpy as np
import numpy.random as rnd
import tensorflow as tf
import os

/home/bodi44/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%run Dino_Classes.ipynb

Constants

In [3]:
width = 80
height = 80
len_epoch = 100000000
num_actions = len(Environment.actions)

Args

In [4]:
checkpoint_nr = None
checkpoint_hz = 120
refresh_hz = 100
update_target_network_hz = 20
training = True
visualize = True
# log_dir = './trained-model/'
log_dir = './logs/'

In [5]:
def check_path_validity():
    """returns -1 if an unvalid path was given."""

    if training and os.path.exists(log_dir):
        print("PATH FOR STORING RESULTS ALREADY EXISTS - Results would be overwritten.")
        return -1

    elif not training and not os.path.exists(log_dir):
        print("PATH DOES NOT EXISTS. TRAINED MODEL NOT FOUND.")
        return -1

    return 0

In [6]:
def play(agent, env, preprocessor):
    # load pretrained model,
    # will fail if the given path doesn't hold a valid model
    name = log_dir + "rex.ckpt"
    if checkpoint_nr is not None:
        name = name + "-" + str(checkpoint_nr)

    agent.load(name)
    agent.explore_prob = 0.0

    while True:
        frame, _, crashed = env.start_game()
        frame = preprocessor.process(frame)
        state = preprocessor.get_initial_state(frame)

        while not crashed:
            action, _  = agent.act(state)
            next_frame, reward, crashed = env.do_action(action)
            print("action: {}".format(env.actions[action]))
            next_frame = preprocessor.process(next_frame)
            next_state = preprocessor.get_updated_state(next_frame)

            state = next_state

        print("Crash")

In [7]:
def train(agent, env, preprocessor):
    agent.update_target_network()

    epoch = 0
    while True:
        epoch += 1
        print("\nEpoch: ", epoch)

        frame, _ , crashed = env.start_game()
        frame = preprocessor.process(frame)
        state = preprocessor.get_initial_state(frame)
        ep_steps, ep_reward = 0, 0

        while not crashed:

            action, explored = agent.act(state)
            next_frame, reward, crashed = env.do_action(action)
            #'*'action if it was randomly chosen (not produced by network)
            action_str = Environment.actions[action] + ["", "*"][explored]
            print("action: {}\t crashed: {}".format(action_str, crashed))
            next_frame = preprocessor.process(next_frame)
            next_state = preprocessor.get_updated_state(next_frame)
            agent.remember(state, action, reward, next_state, crashed)

            ep_steps += 1
            ep_reward += reward

            state = next_state

        agent.replay(epoch)
        agent.explore_less()

        if epoch % update_target_network_hz == 0:
            agent.update_target_network()

        if epoch % checkpoint_hz == 0:
            agent.save(epoch)
            time.sleep(5)

        if epoch % refresh_hz == 0:
            env.refresh_game()


In [8]:
def main(log_dir):
    log_dir = log_dir if log_dir.endswith('/') else log_dir + '/'
    # Make a new directory to store checkpoints and tensorboard summaries,
    if check_path_validity() == -1:
        exit(1)
    elif training and not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Setup tensorflow and tensorboard writers
    session = tf.Session()
    writer = tf.summary.FileWriter(log_dir, session.graph) if visualize else None

    # Initialize key objects: environment, agent and preprocessor
    env = Environment("127.0.0.1", 9090)
    agent = DDQNAgent(session, num_actions, width, height, log_dir, writer)
    preprocessor = Preprocessor(width, height)

    if training:
        train(agent, env, preprocessor)
    else:
        play(agent, env, preprocessor)

In [9]:
main(log_dir)
#python3 -m http.server  8000

PATH FOR STORING RESULTS ALREADY EXISTS - Results would be overwritten.

Epoch:  1


/home/bodi44/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: FORTH*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: FORTH*	 crashed: False
action: UP*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: UP*	 crashed: False
action: DOWN*	 crashed: False
action: UP*	 crashed: False
action: DOWN*	 crashed: False
action: FORTH*	 crashed: False
action: FORTH*	 crashed: False
action: FORTH*	 crashed: False
action: FORTH*	 crashed: True

Epoch:  2
action: FORTH*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: UP*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: UP*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: 

action: DOWN*	 crashed: False
action: UP*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: FORTH*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: True
...Training...

Epoch:  9
action: DOWN*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: UP*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: DOWN*	 crashed: False
action: UP*	 crashed: False
action: FORTH*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: False
action: FORTH*	 crashed: False
action: UP*	 crashed: False
action: UP*	 crashed: False
action: DOWN*	 crashed: False
action: FORTH*	 crashed: False
action: DOWN*	 crashed: True
...Training...

Epoch: 

KeyboardInterrupt: 